In [ ]:
# !pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import joblib
import utils # contient toutes les fonctions crees pour le projet..


from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

### Importation des donnees


In [ ]:
# ROOT = "/content/drive/Shareddrives/Canvas ECo energy/Eco Energy Energy Live Capstone/datasets/" # chemin sur colab
ROOT = "C:/Users/georg/OneDrive/Documents/GitHub/CapstoneProjectEnergy/Dataset/" # chemin en local


dred = "DRED.csv"
qud = "QUD.csv"
sim = "SimDataset.csv"


rw_dred = pd.read_csv(ROOT+dred, names= ["occupancy", "P", "Pn", "P+t1", "P-t1","class_state"], header=None)
rw_qud = pd.read_csv(ROOT+qud, names= ["occupancy", "P", "Pn", "P+t1", "P-t1","class_state"], header=None)
rw_sim = pd.read_csv(ROOT+sim, names= ['Occupancy' , 'Appliance_ID' , 'Sin(time)' , 'Cos(time)' , 'Sin(day)' , 'Cos(day)' , 'Power_consumption_P(t)' , 'Normalized-power-consumption ' , 'P(t)-P(t+1)' , 'P(t)-P(t-1)' , 'Micro-moment_class'], header=None)



In [ ]:
rw_dred.head()

In [ ]:
rw_qud.head()

### Machine Learning

In [ ]:
# !pip install xgboost

#### Normalisation avec le coefficient de variance pour le ML

Qud

In [ ]:
rw_qud.shape , rw_qud.shape[0]-3
input_qud, output_qud = utils.slide_window(dataset=rw_qud ,window_size=3)
input_qud = input_qud.squeeze()

new_qud_dataset = np.concatenate((input_qud, np.expand_dims(output_qud, axis=-1)) , axis = 1)

df_qud = pd.DataFrame(new_qud_dataset, columns = ['oc1','oc2','oc3','p1','p2','p3','pt01','pt02','pt03' ,'pt11','pt22','pt33' ,'cl1','cl2','cl3', 'out' ] )

## normalisons avec le coefficient de variance

df_qud['mean'] = (df_qud['p1'] + df_qud['p2'] + df_qud['p3'] ) / 3 # moyenne des valeurs power pour une window
df_qud['std_window'] = df_qud[['p2' , 'p2' , 'p3']].std(axis=1) # std de P1, P2, P3
df_qud['norm_window'] = ( df_qud['std_window']/df_qud['mean'] ) * ((df_qud[['p1' , 'p2' , 'p3']].max(axis=1) - df_qud[['p1' , 'p2' , 'p3']].min(axis=1)) /1000) # normalization de P1, P2, P3 

## visualisation

debut = 0
fin   = -1

plt.plot(df_qud['p2'][debut:fin])
plt.xlabel('Timestep')
plt.ylabel("Power normalisee")
plt.title("Power dans chaque fenetre" , {'fontsize': 9}  )
plt.show()
print()

plt.plot(df_qud['norm_window'][debut:fin])
plt.xlabel('Timestep')
plt.ylabel("Power normalisee")
plt.title("Power normalisee (w) dans chaque fenetre" , {'fontsize': 9}  )
plt.show()
print()


df_qud.head()

In [ ]:
# quelques statistiques

df_qud.describe()

In [ ]:
qud_corr = df_qud.corr()
sns.heatmap(qud_corr)
qud_corr

Dred

In [ ]:
rw_dred.shape , rw_dred.shape[0]-3
input_dred, output_dred = utils.slide_window(dataset=rw_dred ,window_size=3)
input_dred = input_dred.squeeze()

new_dred_dataset = np.concatenate((input_dred, np.expand_dims(output_dred, axis=-1)) , axis = 1)

df_dred = pd.DataFrame(new_dred_dataset, columns = ['oc1','oc2','oc3','p1','p2','p3','pt01','pt02','pt03' ,'pt11','pt22','pt33' ,'cl1','cl2','cl3', 'out' ] )

## normalisaons avec le coefficient de variance

df_dred['mean'] = (df_dred['p1'] + df_dred['p2'] + df_dred['p3'] ) / 3 # moyenne des valeurs power pour une window
df_dred['std_window'] = df_dred[['p2' , 'p2' , 'p3']].std(axis=1) # std de P1, P2, P3
df_dred['norm_window'] = ( df_dred['std_window']/df_dred['mean'] ) * ((df_dred[['p1' , 'p2' , 'p3']].max(axis=1) - df_dred[['p1' , 'p2' , 'p3']].min(axis=1)) /1000) # normalization de P1, P2, P3 

## visualisation

debut = 0
fin   = -1

plt.plot(df_dred['p2'][debut:fin])
plt.xlabel('Timestep')
plt.ylabel("Power normalisee")
plt.title("Power dans chaque fenetre" , {'fontsize': 9}  )
plt.show()
print()

plt.plot(df_dred['norm_window'][debut:fin])
plt.xlabel('Timestep')
plt.ylabel("Power normalisee")
plt.title("Power normalisee (w) dans chaque fenetre" , {'fontsize': 9}  )
plt.show()
print()


df_dred.head(50)

In [ ]:
# quelques statistiques

df_dred.describe()

In [ ]:
dred_corr = df_dred.corr()
sns.heatmap(dred_corr)
df_dred

Models

In [ ]:
# Separation en train et test

scaler = StandardScaler()
# dred
dred_feature = df_dred.iloc[ : , 1:-3 ].drop( 'out' , axis = 1 )
dred_target  = df_dred.iloc[ : , 1:-3 ].out

dred_features_train = dred_feature.iloc[ : int(dred_feature.shape[0]*0.8 ) , : ]
dred_features_test  = dred_feature.iloc[ int(dred_feature.shape[0]*0.8 ) : , : ]  

dred_target_train = dred_target.iloc[ : int(dred_feature.shape[0]*0.8 ) ]
dred_target_test  = dred_target.iloc[ int(dred_feature.shape[0]*0.8 ) : ]  

# dred_features_train = scaler.fit_transform(dred_features_train)
# dred_features_test  = scaler.fit_transform(dred_features_test) 


# qud
qud_feature = df_qud.iloc[ : , 1:-3 ].drop( 'out' , axis = 1 )
qud_target  = df_qud.iloc[ : , 1:-3 ].out

qud_features_train = qud_feature.iloc[ : int(qud_feature.shape[0]*0.8 ) , : ]
qud_features_test  = qud_feature.iloc[ int(qud_feature.shape[0]*0.8 ) : , : ]  

qud_target_train = qud_target.iloc[ : int(qud_feature.shape[0]*0.8 ) ]
qud_target_test  = qud_target.iloc[ int(qud_feature.shape[0]*0.8 ) : ]  

# qud_features_train = scaler.fit_transform(qud_features_train)
# qud_features_test  = scaler.fit_transform(qud_features_test)

In [ ]:
# qud

xgb = XGBClassifier()


xgb.fit(qud_features_train, qud_target_train)
yhat = xgb.predict(qud_features_test)

print(f"Les scores de chaque classe: \n{f1_score(qud_target_test, yhat, average = None)}")

print()
macro = f1_score(qud_target_test, yhat, average = "macro")
print(f"Les scores de ENSEMBLE classe: \n{macro}")

In [ ]:
# dred

xgb = XGBClassifier()


xgb.fit(dred_features_train, dred_target_train)
yhat = xgb.predict(dred_features_test)


print(f"Les scores de chaque classe: \n{f1_score(dred_target_test, yhat, average = None)}")

print()
macro = f1_score(dred_target_test, yhat, average = "macro")
print(f"Les scores de ENSEMBLE classe: \n{macro}")

### Normalisation avec le stateful pour le ML

In [ ]:
# drop de la colonne de la "power normalized"

dred_dropped = rw_dred.drop('Pn', axis=1)
qud_dropped  = rw_qud.drop('Pn', axis=1)

print(dred_dropped.head())
print()
print(qud_dropped.head())

In [ ]:
processor = utils.DataProcessing(qud_dropped)
X_qud , y_qud = processor.processing()

In [ ]:
processor = utils.DataProcessing(dred_dropped)
X_dred , y_dred = processor.processing()

Models

In [ ]:
# qud

qud_features_train_sf = X_qud[ : int(X_qud.shape[0]*0.8 ) , : ]
qud_features_test_sf  = X_qud[ int(X_qud.shape[0]*0.8 ) : , : ]  

qud_target_train_sf = y_qud[ : int(y_qud.shape[0]*0.8 ) ]
qud_target_test_sf  = y_qud[ int(y_qud.shape[0]*0.8 ) : ]  

# print(qud_features_train_sf)
# print()
# print(qud_target_train_sf)


xgb_sf = XGBClassifier()


xgb.fit(qud_features_train_sf, qud_target_train_sf)
yhat = xgb.predict(qud_features_test_sf)


print(f"Les scores de chaque classe: \n{f1_score(qud_target_test_sf, yhat, average = None)}")

print()
macro = f1_score(qud_target_test, yhat, average = "macro")
print(f"Les scores de ENSEMBLE classe: \n{macro}")

In [ ]:
# dred
 
dred_features_train_sf = X_dred[ : int(X_dred.shape[0]*0.8 ) , : ]
dred_features_test_sf  = X_dred[ int(X_dred.shape[0]*0.8 ) : , : ]  

dred_target_train_sf = y_dred[ : int(y_dred.shape[0]*0.8 ) ]
dred_target_test_sf  = y_dred[ int(y_dred.shape[0]*0.8 ) : ]  

# print(dred_features_test_sf.shape)
# print()
# print(dred_target_test_sf.shape)



xgb_sf = XGBClassifier()


xgb.fit(dred_features_train_sf, dred_target_train_sf)
yhat = xgb.predict(dred_features_test_sf)


print(f"Les scores de chaque classe: \n{f1_score(dred_target_test_sf, yhat, average = None)}")

print()
# print(dred_target_test.shape)
# print(yhat.shape)
macro = f1_score(dred_target_test_sf, yhat, average = "macro")
print(f"Les scores de ENSEMBLE classe: \n{macro}")

Sauvegarde du meilleur model en ML

In [ ]:
# avec pickle

filename = '/Models/best2_model.sav'
pickle.dump(xgb, open(filename, 'wb'))

# avec joblib

filename = 'Models/best1_model.sav'
joblib.dump(xgb, filename)


### Deep Machine Learning

In [ ]:
def create_model(input_shape, output_shape):

  input  = tf.keras.Input( shape = input_shape )
  output = tf.keras.layers.Dense( 128 , activation = "relu" )( input )
  output = tf.keras.layers.Dense( 256 , activation = "relu" )( input )
  lstm   = tf.keras.layers.LSTM(256)( output ) 
  output = tf.keras.layers.Dense( 256 , activation = "relu" )( lstm )
  output = tf.keras.layers.Dense( 256 , activation = "relu" )( output )
  output = tf.keras.layers.Dense(  5  , activation = "softmax" )( output )

  model = tf.keras.Model( inputs= input , outputs= output )

  return model

model = create_model( 
    input_shape = (3, 4), # 3 lectures pour 4 features chacune
    output_shape = 2
 )
model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = "categorical_crossentropy",
    metrics = [utils.get_f1, 'accuracy']
)

In [ ]:
# donnees

rw_dred_cleaned = rw_dred.drop(['Pn'] , axis=1)
rw_qud_cleaned  = rw_qud.drop(['Pn'] , axis=1)

## dred
dred_train = rw_dred_cleaned[:int(df_dred.shape[0]*0.8)]
dred_test = rw_dred_cleaned[int(df_dred.shape[0]*0.8):]
print(dred_train.head())

## qud
qud_train = rw_qud_cleaned[:int(df_qud.shape[0]*0.8)]
qud_test = rw_qud_cleaned[int(df_qud.shape[0]*0.8):]
print(qud_train.head())

In [ ]:
#dred

generateur_train = utils.Generateur(dred_train, window_size=3, batch_size=256)
generateur_test = utils.Generateur(dred_test, window_size=3, batch_size=256)

# print((generateur_train[0]))
dred_hist = model.fit(
    x = generateur_train,
    epochs=10,
    validation_data = generateur_test
)

In [ ]:
#qud

generateur_train = utils.Generateur(qud_train, window_size=3, batch_size=256)
generateur_test = utils.Generateur(qud_test, window_size=3, batch_size=256)

# print((generateur_train[0]))
qud_hist = model.fit(
    x = generateur_train,
    epochs=10,
    validation_data = generateur_test
)